<a href="https://colab.research.google.com/github/jlamorar/scrappinnov19damian/blob/main/PEREZ_DAMIAN_Analizando_EA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Programación para Análisis de Datos

- **Curso**: Programación para Análisis de Datos
- **Docente**: Andrés Palacio
- **Estudiantes**:
  - Damian Jesus Perez Salvatierra -- PREICA2402B020101
  - Jimy Antonio Mora Russy -- PREICA2402B020101
  - Elizabeth Alzate Murillo -- PREICA2402B020101
  - Yilma Valeria Espitia Sanabria -- PREICA2402B020101
- **Fecha**: 03 de Noviembre del 2024
- Institución Universitaria Digital de Antioquia (IUdigital)

---

##Introducción

La recopilación y análisis de datos se han convertido en componentes fundamentales para la toma de decisiones en diversas situaciones; en este caso, en el sector del comercio electrónico. Este análisis es especialmente relevante, ya que los consumidores buscan información detallada sobre productos, precios y características. En este contexto, la utilización de estas técnicas se presenta como una herramienta poderosa para extraer datos de páginas web, permitiendo a los analistas obtener información valiosa de manera eficiente.


Este trabajo explora el uso de técnicas de web scraping, utilizando herramientas como BeautifulSoup, Selenium y Scrapy, para recopilar y analizar datos de una página web específica dedicada a la venta de computadoras, como lo es Mercado Libre.


A lo largo de este análisis, se emplearán estas metodologías para la recolección de datos y se obtendrán insights a partir de los datos extraídos. La finalidad es mostrar cómo estas técnicas pueden no solo facilitar la obtención de información, sino también enriquecer el entendimiento del mercado del producto elegido y las tendencias de consumo.




##Descripción de la página y artículo a analizar

Mercado Libre es una de las plataformas de comercio electrónico más grandes y reconocidas en América Latina, facilitando la compra y venta de una amplia variedad de productos y servicios. Su modelo de negocio permite a los vendedores, desde pequeñas empresas hasta grandes marcas, ofrecer sus productos a una audiencia masiva, mientras que los compradores disfrutan de una experiencia de compra cómoda y accesible. Cuenta con características como la calificación de vendedores, opciones de pago seguras y un sistema de envío eficiente, y así ha logrado establecerse como un líder en el comercio electrónico, adaptándose continuamente a las necesidades del mercado y los consumidores.



La práctica se centrará en analizar los productos de la categoría "portátiles" en el sitio de comercio electrónico. Se utilizará un código en Python para obtener información sobre los productos, incluyendo sus precios, descripciones, calificaciones y otros detalles relevantes. Esta información permitirá entender el contexto de mercado en el que se encuentran estos productos, así como evaluar la competencia y la demanda.


##Descripción del tema de interés que deseas desarrollar en la primera práctica

Para desarrollar la primera actividad nuestro grupo tomó como tema de interés la evaluación del mercado de portátiles en línea, específicamente en la plataforma de Mercado Libre. A través del análisis de las características de los productos ofrecidos, se busca identificar tendencias de precios, la calidad percibida a través de calificaciones y la disponibilidad de garantías. Esto puede ser útil para entender qué factores influyen en la decisión de compra de los consumidores y cómo se posicionan las diferentes marcas en el mercado.Este ejercicio lo llevaremos a cabo por medio de la metodología scraping: Beautifulsoup, Selenium y Scrapy, para conocer cada una de las formas en que este análisis se puede realizar.


##Objetivos:

 * Analizar los precios de los portátiles disponibles en Mercado Libre para entender el rango de precios en el que se encuentran y cómo esto puede influir en las decisiones de compra de los consumidores.


* Comparar las características de los productos, como calificaciones y descripciones, para identificar qué características son valoradas por los consumidores y cómo se diferencia cada producto en el mercado.


* Evaluar la efectividad del uso de técnicas de web scraping para obtener datos relevantes del comercio electrónico, y cómo esto puede ser aplicado en investigaciones de mercado o análisis de competencia.



## Metodo con BeautifulSoup

In [ ]:
import requests # Importamos la librería requests para realizar solicitudes HTTP
from bs4 import BeautifulSoup # Importamos BeautifulSoup de bs4 para analizar documentos HTML y XML
import pprint # Importamos pprint para imprimir estructuras de datos de manera legible

# Función para realizar la búsqueda de un producto y obtener las primeras 10 URLs de resultados
def obtener_urls_productos(busqueda, cantidad=10):
    try:
        # Construimos la URL de búsqueda con el término proporcionado
        url_busqueda = f"https://listado.mercadolibre.com.co/{busqueda}"

        response = requests.get(url, timeout=10)
        response.raise_for_status()

        # Enviamos una solicitud GET a la URL de búsqueda
        response = requests.get(url_busqueda)

        # Guardamos el contenido HTML para inspección
        with open("pagina_portatil.html", "w", encoding='utf-8') as file:
          file.write(response.text)


        # Analizamos el contenido HTML de la respuesta
        soup = BeautifulSoup(response.content, 'html.parser')

        # Encontramos los elementos de la lista de resultados de búsqueda
        items = soup.find_all('li', class_='ui-search-layout__item', limit=cantidad)
        # Obtenemos los enlaces de los productos
        urls = [item.find('a', href=True)['href'] for item in items if item.find('a', href=True)]

        # Retornamos la lista de URLs
        return urls
    except requests.exceptions.RequestException as e:
        print(f"Error al realizar la solicitud: {e}")
        return []

# Función para obtener los detalles de un producto
def obtener_detalles_producto(url_producto):
    try:
        # Enviamos una solicitud GET a la URL del producto
        response = requests.get(url_producto)
        # Analizamos el contenido HTML de la página del producto
        soup = BeautifulSoup(response.content, 'html.parser')

        # Creamos un diccionario para almacenar los datos del producto
        data = {}

        # Extraemos la categoría del producto
        category = soup.find('div', class_='ui-pdp-breadcrumb')
        data['Categoria'] = category.get_text(strip=True) if category else None

        # Extraemos el título del producto
        title = soup.find('h1', class_='ui-pdp-title')
        data['Titulo'] = title.get_text(strip=True) if title else None

        # Extraemos el precio del producto
        price_div = soup.find('div', class_='ui-pdp-price__second-line')
        price = price_div.find('span', class_='andes-money-amount__fraction') if price_div else None
        data['Precio'] = price.get_text(strip=True) if price else None

        # Buscamos un elemento que indique si hay un descuento
        discount = soup.find('s', {'role': 'img', 'aria-label': lambda x: x and x.startswith('Antes:')})
        data['Descuento'] = bool(discount)

        # Extraemos el nombre del vendedor
        seller_button = soup.find('button', class_='ui-pdp-seller__link-trigger-button non-selectable')
        seller = seller_button.find_all('span')[1].get_text(strip=True) if seller_button else None
        data['Vendedor'] = seller

        # Extraemos la calificación promedio del producto
        rating = soup.find('span', {'aria-hidden': 'true', 'class': 'ui-pdp-review__rating'})
        data['Calificacion promedio'] = rating.get_text(strip=True) if rating else None

        # Extraemos la cantidad de calificaciones
        reviews_count = soup.find('span', {'aria-hidden': 'true', 'class': 'ui-pdp-review__amount'})
        data['Cantidad de Calificaciones'] = reviews_count.get_text(strip=True) if reviews_count else None

        # Extraemos la garantía del producto
        warranty = soup.find('p', class_='ui-pdp-family--REGULAR ui-pdp-media__title', string=lambda text: text and text.endswith('garant√≠a de f√°brica.'))
        data['Garantia'] = warranty.get_text(strip=True) if warranty else None

        # Extraemos la descripción del producto
        description = soup.find('p', class_='ui-pdp-description__content')
        data['Descripcion'] = description.get_text(strip=True) if description else None

        # Extraemos información de stock
        stock_info = soup.find('p', class_='ui-pdp-stock-information__title')
        data['Stock'] = stock_info.get_text(strip=True) if stock_info else None

        # Extraemos la cantidad total de opiniones
        total_opinions = soup.find('span', class_='total-opinion')
        data['Cantidad de Opiniones'] = total_opinions.get_text(strip=True) if total_opinions else None

        # Extraemos el número de publicación del producto
        publication_number = soup.find('span', class_='ui-pdp-color--BLACK ui-pdp-family--SEMIBOLD')
        data['Numero de Publicacion'] = publication_number.get_text(strip=True) if publication_number else None

        # Agregamos la URL del producto al diccionario
        data['URL del Producto'] = url_producto

        # Retornamos el diccionario con los datos del producto
        return data
    except Exception as e:
        print(f"Error al procesar un producto: {e}")
        return {}

# Realizamos la búsqueda y obtenemos los detalles de los productos
busqueda = 'portatil'
# Primero, obtenemos las URLs de los primeros 10 productos encontrados
urls_productos = obtener_urls_productos(busqueda)

# Lista para almacenar los detalles de cada producto
detalles_productos = []

# Iteramos sobre cada URL y obtenemos los detalles del producto
for url in urls_productos:
    detalles_producto = obtener_detalles_producto(url)
    detalles_productos.append(detalles_producto)

# Finalmente, imprimimos los detalles de todos los productos de manera legible
pprint.pprint(detalles_productos)


## Metodo con Selenium

In [ ]:
# En caso de que no tengamos gdown o selenium instalado, procedemos a descargarlo mediante pip
!pip install gdown
!pip install selenium

# Primeramente descargamos el archivo ChromeDriver desde Google Drive
!gdown --id 1w78mIreuYtixvsmVcgdKLP_mAy4KxMzv

# Luego cambiamos el permiso para executar el chromedriver
!chmod +x /content/chromedriver
# Luego movemos el archivo chromedriver al directorio adecuado
!mv /content/chromedriver /usr/local/bin/chromedriver

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pprint
import traceback  # Importamos traceback para obtener detalles completos del error


# Configuramos opciones para Chrome en modo headless
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Inicializamos el controlador de Chrome con las opciones configuradas
driver = webdriver.Chrome(options=chrome_options)

# Función para realizar la búsqueda de un producto y obtener las URLs de los primeros 10 resultados
def obtener_urls_productos(busqueda, cantidad=10):
    # Construimos la URL de búsqueda con el término proporcionado
    url_busqueda = f"https://listado.mercadolibre.com.co/{busqueda}"
    driver.get(url_busqueda)
    try:
        # Esperamos hasta que los elementos de la lista de resultados estén presentes
        productos = WebDriverWait(driver, 15).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li.ui-search-layout__item a'))
        )

        # Obtenemos las URLs de los primeros 10 productos
        urls = [producto.get_attribute('href') for producto in productos[:cantidad]]
        return urls
    except Exception as e:
        print(f"Error al obtener las URLs de los productos: {e}")
        traceback.print_exc()  # Imprime el traceback completo del error
        return []

# Función para obtener los detalles de un producto
def obtener_detalles_producto(url_producto):
    data = {}
    try:
        driver.get(url_producto)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-pdp-title')))

        # Extraemos los detalles del producto usando bloques try-except
        try:
            data['Categoria'] = driver.find_element(By.CLASS_NAME, 'ui-pdp-breadcrumb').text.strip()
        except:
            data['Categoria'] = None

        try:
            data['Titulo'] = driver.find_element(By.CLASS_NAME, 'ui-pdp-title').text.strip()
        except:
            data['Titulo'] = None

        try:
            data['Precio'] = driver.find_element(By.CSS_SELECTOR, 'div.ui-pdp-price__second-line span.andes-money-amount__fraction').text.strip()
        except:
            data['Precio'] = None

        try:
            discount = driver.find_element(By.CSS_SELECTOR, 's[role="img"][aria-label^="Antes:"]')
            data['Descuento'] = True if discount else False
        except:
            data['Descuento'] = False

        try:
            seller_button = driver.find_element(By.CLASS_NAME, 'ui-pdp-seller__link-trigger-button')
            seller = seller_button.find_elements(By.TAG_NAME, 'span')[1].text.strip()
            data['Vendedor'] = seller
        except:
            data['Vendedor'] = None

        try:
            data['Calificacion promedio'] = driver.find_element(By.CSS_SELECTOR, 'span.ui-pdp-review__rating[aria-hidden="true"]').text.strip()
        except:
            data['Calificacion promedio'] = None

        try:
            data['Cantidad de Calificaciones'] = driver.find_element(By.CSS_SELECTOR, 'span.ui-pdp-review__amount[aria-hidden="true"]').text.strip()
        except:
            data['Cantidad de Calificaciones'] = None

        try:
            warranty = driver.find_element(By.XPATH, "//p[contains(., 'garantía de fábrica.')]")
            data['Garantia'] = warranty.text.strip() if 'garantía de fábrica.' in warranty.text.strip() else None
        except:
            data['Garantia'] = None

        try:
            data['Descripcion'] = driver.find_element(By.CLASS_NAME, 'ui-pdp-description__content').text.strip()
        except:
            data['Descripcion'] = None

        try:
            data['Stock'] = driver.find_element(By.CLASS_NAME, 'ui-pdp-stock-information__title').text.strip()
        except:
            data['Stock'] = None

        try:
            data['Cantidad de Opiniones'] = driver.find_element(By.CLASS_NAME, 'total-opinion').text.strip()
        except:
            data['Cantidad de Opiniones'] = None

        try:
            data['Numero de Publicacion'] = driver.find_element(By.CSS_SELECTOR, 'span.ui-pdp-color--BLACK.ui-pdp-family--SEMIBOLD').text.strip()
        except:
            data['Numero de Publicacion'] = None

        # Agregamos la URL del producto al diccionario
        data['URL del Producto'] = url_producto

    except Exception as e:
        print(f"Error al extraer los datos del producto: {e}")
    return data

# Realizamos la búsqueda y obtenemos los detalles de los primeros 10 productos
busqueda = 'portatil'
urls_productos = obtener_urls_productos(busqueda)

# Lista para almacenar los detalles de cada producto
detalles_productos = []

# Iteramos sobre cada URL y obtenemos los detalles del producto
for url in urls_productos:
    detalles_producto = obtener_detalles_producto(url)
    detalles_productos.append(detalles_producto)

# Finalmente, imprimimos los detalles de todos los productos de manera legible
pprint.pprint(detalles_productos)

# Cerramos el navegador al final
driver.quit()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 30.9 MB/s eta 0:00:00
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1w78mIreuYtixvsmVcgdKLP_mAy4KxMzv
To: /content/chromedriver
100% 17.3M/17.3M [00:00<00:00, 33.1MB/s]
Error al extraer los datos del producto: Message: 
Stacktrace:
#0 0x55831ab9210a <unknown>
#1 0x55831a6ad460 <unknown>
#2 0x55831a6f9f96 <unknown>
#3 0x55831a6fa221 <unknown>
#4 0x55831a73f5d4 <unknown>
#5 0x55831a71e03d <unknown>
#6 0x55831a73caf6 <unknown>
#7 0x55831a71ddb3 <unknown>
#8 0x55831a6ecc10 <unknown>
#9 0x55831a6edbee <unknown>
#10 0x55831ab5e39b <unknown>
#11 0x55831ab62338 <unknown>
#12 0x55831ab4bebc <unknown>
#13 0x55831ab62eb7 <unknown>
#14 0x558

## Metodo con Scrapy

### Nota
El metodo de scrappy solo puede correr una sola vez, en caso de correrlo una segunda vez es necesario abortar el runtime y correrlo nuevamente

In [ ]:
# Descargamos la librería Scrapy en caso de que no la tengamos
!pip install scrapy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.2/254.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 5.0 MB/s eta 0:00:00


In [ ]:
from twisted.internet import reactor  # Importamos el reactor de Twisted, necesario para controlar el ciclo de eventos de Scrapy
from scrapy.crawler import CrawlerRunner  # Importamos CrawlerRunner para gestionar el ciclo de vida del spider en Scrapy
from scrapy.utils.log import configure_logging  # Importamos configure_logging para configurar el sistema de logging de Scrapy
import scrapy  # Importamos la biblioteca Scrapy que usaremos para crear el spider
import pprint  # Importamos pprint para imprimir los datos de forma más legible

class MercadolibreSpider(scrapy.Spider):
    name = 'mercadolibre_spider'
    allowed_domains = ['mercadolibre.com.co']
    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    start_urls = ['https://listado.mercadolibre.com.co/portatil']
    cantidad_productos = 10  # Definimos la cantidad de productos a obtener

    def parse(self, response):
        # Seleccionamos los enlaces de los primeros 'cantidad_productos' resultados de la búsqueda
        productos = response.css('li.ui-search-layout__item a::attr(href)').getall()[:self.cantidad_productos]

        # Iteramos sobre cada URL de producto y hacemos una solicitud a cada uno
        for producto_url in productos:
            yield response.follow(producto_url, self.parse_product)

    def parse_product(self, response):
        data = {}

        try:
            data['Categoria'] = response.css('div.ui-pdp-breadcrumb::text').get()
        except:
            data['Categoria'] = None

        try:
            data['Titulo'] = response.css('h1.ui-pdp-title::text').get()
        except:
            data['Titulo'] = None

        try:
            data['Precio'] = response.css('div.ui-pdp-price__second-line span.andes-money-amount__fraction::text').get()
        except:
            data['Precio'] = None

        try:
            data['Descuento'] = bool(response.css('s[role="img"][aria-label^="Antes:"]'))
        except:
            data['Descuento'] = None

        try:
            vendedor_data = response.css('button.ui-pdp-seller__link-trigger-button span::text').getall()
            data['Vendedor'] = vendedor_data[1] if len(vendedor_data) > 1 else None
        except:
            data['Vendedor'] = None

        try:
            data['Calificacion promedio'] = response.css('span.ui-pdp-review__rating[aria-hidden="true"]::text').get()
        except:
            data['Calificacion promedio'] = None

        try:
            data['Cantidad de Calificaciones'] = response.css('span.ui-pdp-review__amount[aria-hidden="true"]::text').get()
        except:
            data['Cantidad de Calificaciones'] = None

        try:
            data['Garantia'] = response.xpath("//p[contains(., 'garantía de fábrica.')]/text()").get()
        except:
            data['Garantia'] = None

        try:
            data['Descripcion'] = response.css('p.ui-pdp-description__content::text').get()
        except:
            data['Descripcion'] = None

        try:
            data['Stock'] = response.css('p.ui-pdp-stock-information__title::text').get()
        except:
            data['Stock'] = None

        try:
            data['Cantidad de Opiniones'] = response.css('span.total-opinion::text').get()
        except:
            data['Cantidad de Opiniones'] = None

        try:
            data['Numero de Publicacion'] = response.css('span.ui-pdp-color--BLACK.ui-pdp-family--SEMIBOLD::text').get()
        except:
            data['Numero de Publicacion'] = None

        data['URL del Producto'] = response.url

        # Imprimimos los datos del producto
        pprint.pprint(data)


# Configuración de logging
configure_logging()

# Creación de instancia de CrawlerRunner
runner = CrawlerRunner()

# Verificamos si el reactor ya está en ejecución antes de iniciarlo
if not reactor.running:
    # Ejecutamos el spider MercadolibreSpider
    d = runner.crawl(MercadolibreSpider)
    # Detenemos el reactor cuando el spider termina
    d.addBoth(lambda _: reactor.stop())
    # Iniciamos el ciclo de eventos del reactor
    reactor.run()


INFO:scrapy.addons:Enabled addons:
[]
2024-11-01 01:13:24 [scrapy.addons] INFO: Enabled addons:
[]
/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)
INFO:scrapy.extensions.telnet:Telnet Password: abbde5c4b69c3c7e
2024-11-01 01:13:24 [scrapy.extensions.telnet] INFO: Telnet Password: abbde5c4b69c3c7e
INFO:scrapy.middleware:Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage

{'Calificacion promedio': '4.8',
 'Cantidad de Calificaciones': '(55)',
 'Cantidad de Opiniones': '16 comentarios',
 'Categoria': None,
 'Descripcion': 'Aviso legal',
 'Descuento': True,
 'Garantia': '12 meses de garantía de fábrica.',
 'Numero de Publicacion': '24 GB',
 'Precio': '1.779.309',
 'Stock': 'Stock disponible',
 'Titulo': 'Potátil V-Series V14 14" color gris 24GB de Ram - 512GB SSD - '
           'Intel Core i5',
 'URL del Producto': 'https://www.mercadolibre.com.co/potatil-v-series-v14-14-color-gris-24gb-de-ram-512gb-ssd-intel-core-i5/p/MCO36489354',
 'Vendedor': 'DISTRIMAK '}
{'Calificacion promedio': '4.7',
 'Cantidad de Calificaciones': '(442)',
 'Cantidad de Opiniones': '127 comentarios',
 'Categoria': None,
 'Descripcion': 'Aviso legal',
 'Descuento': True,
 'Garantia': '12 meses de garantía de fábrica.',
 'Numero de Publicacion': '16 GB',
 'Precio': '1.739.900',
 'Stock': 'Stock disponible',
 'Titulo': 'Portatil Asus E1504fa-nj474 Ryzen 5-7520u Ram 16gb Ssd 512gb Col

DEBUG:scrapy.core.engine:Crawled (200) <GET https://articulo.mercadolibre.com.co/MCO-1133597253-laptop-hp-240-g8-intel-celeron-n4120-8gb-256gb-ssd-windows10-_JM#is_advertising=true&position=1&search_layout=stack&type=pad&tracking_id=8e00f9c3-a277-4dd0-a7fb-f62c6db47db4&is_advertising=true&ad_domain=VQCATCORE_LST&ad_position=1&ad_click_id=NWYzNGNiMzQtYWE4NS00Nzc1LWI1NWQtMThiZjZkNGM2MjYx> (referer: https://listado.mercadolibre.com.co/portatil)
2024-11-01 01:13:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://articulo.mercadolibre.com.co/MCO-1133597253-laptop-hp-240-g8-intel-celeron-n4120-8gb-256gb-ssd-windows10-_JM#is_advertising=true&position=1&search_layout=stack&type=pad&tracking_id=8e00f9c3-a277-4dd0-a7fb-f62c6db47db4&is_advertising=true&ad_domain=VQCATCORE_LST&ad_position=1&ad_click_id=NWYzNGNiMzQtYWE4NS00Nzc1LWI1NWQtMThiZjZkNGM2MjYx> (referer: https://listado.mercadolibre.com.co/portatil)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://articulo.mercadolibre.com.co/MCO-

{'Calificacion promedio': '4.7',
 'Cantidad de Calificaciones': '(302)',
 'Cantidad de Opiniones': '104 comentarios',
 'Categoria': None,
 'Descripcion': 'Convierte las tareas cotidianas en algo especial con Vivobook '
                '15, tu herramienta esencial para hacer las cosas más '
                'fácilmente, en cualquier lugar. También es completamente '
                'fácil de usar, con su bisagra plana de 180° y el protector '
                'físico de la cámara web. ASUS Antimicrobial Guard Plus '
                'protege las superficies que se tocan con frecuencia de '
                'bacterias dañinas, salvaguardando su salud. ¡Haz que todos '
                'tus días sean más agradables con Vivobook 15!',
 'Descuento': True,
 'Garantia': '12 meses de garantía de fábrica.',
 'Numero de Publicacion': '20 GB',
 'Precio': '2.021.137',
 'Stock': 'Stock disponible',
 'Titulo': 'Portatil Asus Vivobook X1504za-nj372 Corei5 1235u 20gb 512gb Azul ',
 'URL del Producto': 'htt

INFO:scrapy.core.engine:Closing spider (finished)
2024-11-01 01:13:29 [scrapy.core.engine] INFO: Closing spider (finished)
INFO:scrapy.statscollectors:Dumping Scrapy stats:
{'downloader/request_bytes': 9009,
 'downloader/request_count': 13,
 'downloader/request_method_count/GET': 13,
 'downloader/response_bytes': 1152680,
 'downloader/response_count': 13,
 'downloader/response_status_count/200': 10,
 'downloader/response_status_count/301': 1,
 'downloader/response_status_count/302': 2,
 'dupefilter/filtered': 1,
 'elapsed_time_seconds': 3.947109,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2024, 11, 1, 1, 13, 29, 558319, tzinfo=datetime.timezone.utc),
 'httpcompression/response_bytes': 6564543,
 'httpcompression/response_count': 10,
 'log_count/DEBUG': 16,
 'log_count/INFO': 10,
 'memusage/max': 194367488,
 'memusage/startup': 194367488,
 'request_depth_max': 1,
 'response_received_count': 10,
 'scheduler/dequeued': 13,
 'scheduler/dequeued/memory': 13,
 'scheduler/

{'Calificacion promedio': '4.6',
 'Cantidad de Calificaciones': '(383)',
 'Cantidad de Opiniones': '132 comentarios',
 'Categoria': None,
 'Descripcion': 'Sistema operativo: FreeDos (WINDOWS 10 PRE-INSTALADO)',
 'Descuento': True,
 'Garantia': '12 meses de garantía de fábrica.',
 'Numero de Publicacion': 'NEW TECNOLOGIES',
 'Precio': '897.900',
 'Stock': None,
 'Titulo': 'Laptop Hp 240 G8 Intel Celeron N4120 8gb 256gb Ssd Windows10',
 'URL del Producto': 'https://articulo.mercadolibre.com.co/MCO-1133597253-laptop-hp-240-g8-intel-celeron-n4120-8gb-256gb-ssd-windows10-_JM',
 'Vendedor': 'NEW TECNOLOGIES'}
{'Calificacion promedio': '4.0',
 'Cantidad de Calificaciones': '(1)',
 'Cantidad de Opiniones': '1 comentario',
 'Categoria': None,
 'Descripcion': 'PORTÁTIL',
 'Descuento': True,
 'Garantia': None,
 'Numero de Publicacion': '+100',
 'Precio': '524.999',
 'Stock': 'Stock disponible',
 'Titulo': 'Portatil Hp Probook 450 G2 Core I5 4ta 8gb Ssd 240gb '
           '(Reacondicionado)',
 'UR

##Metodología empleada de Scraping

Si bien en el ejercicio realizamos pruebas con las técnicas indicadas, lo que nos permitió trabajar con cada una de ellas y validar su comportamiento y utilidad, elegimos Selenium, ya que es ideal para interactuar con páginas dinámicas donde los datos se cargan dinámicamente. Aunque consume más recursos para su procesamiento, Selenium es útil para realizar pruebas o automatizar la navegación y las acciones de clics. Es flexible para usarse en múltiples navegadores y simula la interacción del usuario.


##Resultados y Conclusiones

Cada una de estas herramientas ofrece ventajas únicas que nos permiten, de diferentes maneras, obtener datos valiosos para el análisis en páginas web. Por ejemplo, BeautifulSoup es ideal para el análisis de contenido HTML, facilitando la extracción de información estructurada de documentos web. Por otro lado, Selenium permite la automatización de navegadores, lo que es útil para interactuar con páginas que requieren acciones dinámicas, como clics y desplazamientos. Scrapy es un framework que facilita la construcción de spiders para la recolección masiva de datos, optimizando el proceso de scraping.



A través de estas técnicas, se pudieron analizar datos dinámicos como precios, disponibilidad y características de los computadores, interactuar con los elementos de la página web, y validar la eficiencia del sitio y sus restricciones. Al mismo tiempo, se identificaron posibles mejoras en el sitio, actuando siempre desde la ética y las políticas de legalidad del mismo.

